In [ ]:
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 huggingface_hub==0.22.2 autotrain-advanced==0.7.77 -qqq
!pip install --upgrade huggingface-hub -qqq

# **SQL 프롬프트**

In [ ]:
def make_prompt(ddl, question, query=''):
  prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

###SQL:
{query}"""

  return prompt

# **평가를 위한 요청 jsonl 작성 함수**

In [ ]:
import json
import pandas as pd
from pathlib import Path

def make_requests_for_gpt_evaluation(df, filename, dir='requests'):
  # 경로 확인시 -> 없으면 생성
  if not Path(dir).exists():
    Path(dir).mkdir(parents=True)

  prompts = []

  # 데이터프레임 읽기
  for idx, row in df.iterrows():
    # GPT 요청 JSON 리스트 생성
    # response_format: GPT가 JSON 형식으로 응답하도록
    prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. If gen_sql and gt_sql do equal job, return "yes" else return "no". Output JSON Format: {"resolve_yn": ""}""" + f"""

DDL: {row['context']}
Question: {row['question']}
gt_sql: {row['answer']}
gen_sql: {row['gen_sql']}"""
)
  # prompts 리스트를 이용하여 GPT 요청 JSON 리스트 생성
  # response_format: -> GPT 응답 데이터 형식
  # message: GPT 시스템 역할 지정 및 평가 요청 내용 포함
  jobs = [{"model": "gpt-4-turbo-preview", "response_format" : { "type": "json_object" }, "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]

  # 쓰기 모드로 파일 열기
  with open(Path(dir, filename), "w") as f:
    for job in jobs:
      # json.dumps -> 파이썬 객체를 JSON 문자열로 변환하는 함수
      json_string = json.dumps(job)
      f.write(json_string + "\n")

# **비동기 요청 명령**

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "자신의 OpenAI API 키 입력"

python api_request_parallel_processor.py \
  --requests_filepath {요청 파일 경로} \
  --save_filepath {생성할 결과 파일 경로} \
  --request_url https://api.opneai.com/v1/chat/completions \
  --max_requests_per_minute 300 \
  --max_tokens_per_minute 100000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20

# **결과 jsonl 파일을 csv로 변환하는 함수**

In [ ]:
def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
  prompts = []
  responses = []

  with open(input_file, 'r') as json_file:
    # 파일을 한줄씩 읽음
    for data in json_file:
      # json.loads(data) -> 한 줄의 JSON 데이터를 파이썬 객체(리스트 또는 딕셔너리)로 변환
      prompts.append(json.loads(data)[0]['message'][0]['content'])
      responses.append(json.loads(data)[1]['choices'][0]['message']['content'])

  # 판다스 데이터프레임 생성
  # 컬럼명: 데이터 형식
  df = pd.DataFrame({prompt_column: prompts, response_column: responses})
  # CSV 파일로 저장
  df.to_csv(output_file, index=False)

  return df